In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys,os
from sklearn.preprocessing import normalize
#import functions
#import plotting_functions
sys.path.append('..')
sys.path.append('data')

%matplotlib inline
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output,display,HTML
np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [3]:
data_dir = '../data'
data_fname = 'atoabsgovhack2017.xlsx'
data_fpath = os.path.join(data_dir,data_fname)
data = pd.read_excel(data_fpath,sheetname='Data')

In [29]:
def print_full(x, print_func, max_rows = None, max_cols = None):
    with pd.option_context('display.max_rows', max_rows, 'display.max_columns', max_cols):
        print_func(x)

In [30]:
df = data.set_index('Postcode')
print_full(df.loc[800,:],display)

,id,Income year,Individuals1,Taxable income or loss1,Net tax,Gross interest,Net rent,Net capital gain,Total income or loss,Total deductions,Salary or wages,Medicare levy,Medicare levy surcharge,Total work related expenses,Gifts or donations,HELP assessment debt2,0-4 years,5-9 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85-89 years,90-94 years,95-99 years,100 years and over,Oceanian,North-West European,Southern and Eastern European,North African and Middle Eastern,South-East Asian,North-East Asian,Southern and Central Asian,People of the Americas,Sub-Saharan African,Male,Female,under or 0,$1 to 1000,1000-1999,2000+,Married in a registered marriage,Married in a de facto marriage,Not married
Postcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
800,200600800,2006,1660,80189786,19291288,982029,-948556,2189379,84768019,4545219,66405766,959934,75734,2822606,170298,260038,62,39,57,111,323,386,263,212,164,181,179,149,133,73,38,42,27,14,4,3,0,553,1018,130,6,67,95,26,4,3,1482,999,85,1114,476,111,333,269,742
800,201100800,2011,3805,217500939,51780253,2319428,-4678343,13759402,224848733,7811365,169930827,2653012,86688,5667879,325092,610288,169,73,98,159,648,858,531,381,322,284,289,254,190,114,67,47,44,9,5,0,0,836,1943,231,35,197,250,143,44,16,3629,2835,181,1512,1343,406,847,588,1480
800,201500800,2015,5579,345853434,83997228,2280912,-4125084,3027094,358517667,12190000,324998792,4778129,236164,9707518,433253,910628,278,145,110,119,658,1278,1058,636,377,440,385,370,276,181,84,42,17,12,4,0,0,874,2523,306,73,295,772,426,48,17,2610,1937,375,1556,1823,1100,1375,799,2001


In [60]:
from sklearn import datasets, linear_model

In [19]:
n = data.loc[0,'Individuals1']
t = data.loc[0,'Taxable income or loss1 ']
mu = t/n

48307.099999999999

measure, postcode, year, 
value, standard deviation, low, high
year from 2006, 2011, 2016, 2021, 2026, 2031
growth assumption for each population for each year

In [117]:
x_year = [
    'Income year',
]
# inflation rate = 0.025, interest rate = 0.015
growth_rates = np.array([0.025,0.015,0.015,0.015,0.025])
x_vars = [
    'Taxable income or loss1 ',
    #'Net tax',
    'Gross interest',
    'Net rent',
    'Net capital gain',
    'Total income or loss',
    #'Total deductions',
    #'Salary or wages',
    #'Medicare levy',
    #'Medicare levy surcharge',
    #'Total work related expenses',
    #'Gifts or donations',
    #'HELP assessment debt2'
]
y_vars = [
 '0-4 years',
 '5-9 years',
 '10-14 years',
 '15-19 years',
 '20-24 years',
 '25-29 years',
 '30-34 years',
 '35-39 years',
 '40-44 years',
 '45-49 years',
 '50-54 years',
 '55-59 years',
 '60-64 years',
 '65-69 years',
 '70-74 years',
 '75-79 years',
 '80-84 years',
 '85-89 years',
 '90-94 years',
 '95-99 years',
 '100 years and over',
 'Oceanian',
 'North-West European',
 'Southern and Eastern European',
 'North African and Middle Eastern',
 'South-East Asian',
 'North-East Asian',
 'Southern and Central Asian',
 'People of the Americas',
 'Sub-Saharan African',
 'Male',
 'Female',
 'under or 0',
 '$1 to 1000',
 '1000-1999',
 '2000+',
 'Married in a registered marriage',
 'Married in a de facto marriage',
 'Not married'
]
normalising_var = ['Individuals1']

In [159]:
for y_var in y_vars:
    for postcode in df.index.unique():
        #y_var = '20-24 years'
        x1 = np.divide(
            df.loc[postcode,x_vars],
            df.loc[postcode,normalising_var]
        ).values
        x2 = df.loc[postcode,x_year]
        X = np.concatenate((x1,x2),axis=1)
        Y = np.reshape(df.loc[postcode,y_var].values,(-1,1))
        regr = linear_model.LinearRegression()
        regr.fit(X, Y)
        final_x_vals = np.divide(
            df.loc[postcode,x_vars].iloc[-1,:].values,
            df.loc[postcode,normalising_var].iloc[-1,:].values
        )
        x1_new = np.array(
            (
                final_x_vals * (growth_rates + 1)**5,
                final_x_vals * (growth_rates + 1)**10,
                final_x_vals * (growth_rates + 1)**15
            )
        )
        x2_new = np.reshape(np.array([2020,2025,2030]),(-1,1))
        x_new = np.concatenate((x1_new,x2_new),axis=1)
        pred = regr.predict(X=x_new)
        print(pred)
        break

[[ 410.01]
 [ 559.4 ]
 [ 728.45]]
[[ 207.7 ]
 [ 278.68]
 [ 359.02]]
[[ 145.16]
 [ 184.94]
 [ 229.93]]
[[ 130.88]
 [ 144.27]
 [ 159.38]]
[[  894.93]
 [ 1162.88]
 [ 1465.91]]
[[ 1832.29]
 [ 2459.48]
 [ 3169.19]]
[[ 1535.1 ]
 [ 2075.12]
 [ 2686.33]]
[[  893.2 ]
 [ 1184.29]
 [ 1513.73]]
[[ 522.5 ]
 [ 687.08]
 [ 873.26]]
[[ 599.67]
 [ 780.37]
 [ 984.88]]
[[ 511.63]
 [ 654.92]
 [ 817.05]]
[[ 509.02]
 [ 666.34]
 [ 844.36]]
[[ 361.15]
 [ 457.52]
 [ 566.58]]
[[ 246.07]
 [ 319.72]
 [ 403.08]]
[[ 113.81]
 [ 147.53]
 [ 185.68]]
[[ 42.56]
 [ 43.19]
 [ 43.89]]
[[ 14.87]
 [ 12.43]
 [  9.65]]
[[ 9.9 ]
 [ 7.53]
 [ 4.85]]
[[ 4.14]
 [ 4.3 ]
 [ 4.49]]
[[-2.14]
 [-4.56]
 [-7.3 ]]
[[ 0.]
 [ 0.]
 [ 0.]]
[[ 1102.55]
 [ 1361.04]
 [ 1653.4 ]]
[[ 3486.68]
 [ 4576.98]
 [ 5810.55]]
[[ 418.88]
 [ 546.6 ]
 [ 691.12]]
[[ 113.65]
 [ 159.66]
 [ 211.73]]
[[ 441.74]
 [ 607.77]
 [ 795.63]]
[[ 1151.48]
 [ 1581.1 ]
 [ 2067.47]]
[[  660.98]
 [  926.96]
 [ 1228.04]]
[[  78.94]
 [ 113.93]
 [ 153.5 ]]
[[ 26.78]
 [ 37.84]
 [ 50.

In [156]:
pred

array([[  894.93],
       [ 1162.88],
       [ 1465.91]])